In [24]:
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d

In [62]:
ds = xr.open_dataset('/Users/asozza/Documents/work/ecearth/data/nemo/initial/woa13-levitus.nc')

In [27]:
domain1 = xr.open_dataset('/Users/asozza/Documents/work/ecearth/data/nemo/domain/eORCA1/domain_cfg.nc')

In [28]:
domain2 = xr.open_dataset('/Users/asozza/Documents/work/ecearth/data/nemo/domain/ORCA2/domain_cfg.nc')

In [30]:
old_depths = domain1['nav_lev'].values
new_depths = domain2['nav_lev'].values

In [74]:
ds = ds.assign_coords(nav_lev=("z", old_depths))

In [75]:
if 'nav_lev' not in ds.coords:
    print("Dataset does not contain 'nav_lev' coordinate.")
else:
    print("Dataset contains 'nav_lev' coordinate.")

Dataset contains 'nav_lev' coordinate.


In [34]:
def vertical_interp(field, old_z, new_z):
    # field: numpy array (..., z, ...)
    interp_func = interp1d(old_z, field, axis=0, bounds_error=False, fill_value="extrapolate")
    return interp_func(new_z)

In [39]:
# Select a single time index (e.g., t=0) to match the shape of old_depths along the interpolation axis
field = interp1d(old_depths, ds['thetao'].isel(time_counter=0).values, axis=0, bounds_error=False, fill_value="extrapolate")(new_depths)

In [52]:
interp_vars = {}
for var in ['thetao', 'so']:
    interp_data = []
    for t in range(ds.sizes['time_counter']):
        slice_t = ds[var].isel(time_counter=t).values
        interp_slice = vertical_interp(slice_t, old_depths, new_depths)
        interp_data.append(interp_slice)
    interp_array = np.stack(interp_data, axis=0)
    dims = ("time_counter", "z", "y", "x")
    coords = {"time_counter": ds["time_counter"], "z": new_depths, "y": ds["y"], "x": ds["x"]}
    interp_vars[var] = xr.DataArray(interp_array, dims=dims, coords=coords, name=var)

new_ds = xr.merge([interp_vars['thetao'], interp_vars['so']])